In [1]:
from bs4 import BeautifulSoup as bs
import requests as req
from pprint import *
bsr = bs(open("sh_map_v1.0.osm"),"lxml")

nodes = bsr.find_all('node')
print(len(nodes))

446258


In [2]:
import pandas as pd
datalist=[]
for node in nodes:
    datalist.append([node['id'],node['lat'],node['lon']])
df1=pd.DataFrame(datalist,columns=['origin_id','lat','lon'])
df1['new_id']=df1.index
df1

,origin_id,lat,lon,new_id
0,29583117,31.1110347,121.3798759,0
1,29583119,31.1130409,121.3817942,1
2,29583122,31.1236993,121.3894210,2
3,29583123,31.1249980,121.3904357,3
4,29583124,31.1286618,121.3943607,4
5,29583125,31.1344869,121.4004563,5
6,29583126,31.1369635,121.4028790,6
7,29583127,31.1379874,121.4039203,7
8,30198387,31.1122153,121.3567958,8
9,30198389,31.1139604,121.3558934,9


In [3]:
print df1.lat.max()
print df1.lat.min()

31.5703252
30.9466593


In [4]:
print df1.lon.max()
print df1.lon.min()

121.8071166
120.8671296


In [5]:
df1.to_csv('Nodes.csv',index=False)

In [20]:
# None sense...
# length=len(df1)
# file=open('WA_Nodes.txt','wb')
# for i in range(length):
#     print>>file,    df1.loc[i,'new_id'],df1.loc[i,'lat'],df1.loc[i,'lon']

# file.close()

In [6]:
ways = bsr.find_all('way')
way_kinds = ['motorway','motorway_link','primary','primary_link',\
            'secondary','secondary_link','tertiary', 'tertiary_link','residential','living_street',\
            'service','trunk','trunk_link','unclassified','road','track']

highways = []
way_types = set()
pc = 0
for w in ways:
    flag=False
    for tag in w.find_all('tag'):
        if tag['k']=='highway':
            if tag['v'] in way_kinds:
                flag=True
                break
            else:
                if tag['v'] not in way_types:
#                     print w
                    pc += 1
                way_types.add(tag['v'])
                
    if flag:
        highways.append(w)
# highways[:5]
print way_types

set(['bus_stop', 'bus_guideway', 'proposed', 'footway', 'platform', 'cycleway', 'pedestrian', 'construction', 'steps', 'bridleway', 'crossing', 'path', 'yes', 'raceway'])


In [7]:
import pandas as pd
from collections import defaultdict
segMap = defaultdict(int)

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n-1):
        yield l[i:i+n]
final_list=[]
for w in highways:

    isOneway = False
    category = 'unclassified'
    for tag in w.find_all('tag'):
        if tag['k']=='oneway' and tag['v'] =='yes':
            isOneway = True
        if tag['k']=='highway':
            category = tag['v']

    nodes=[nd['ref'] for nd in w.find_all('nd')]
    for node in nodes:
        segMap[node] += 1
    len_nodes=len(nodes)
   
    
    if  isOneway == False:
        

#         nodes_list=list(chunks(nodes,4))
#         if(len(nodes_list[-1])==1):
#             nodes_list.pop()
            
#         for nds in nodes_list:
        final_list.append([1, False,  category,nodes])
        final_list.append([1, False,  category,nodes[::-1]])
        

    else:
        final_list.append([1, True,  category,nodes])
#         nodes_list=list(chunks(nodes,4))
#         if(len(nodes_list[-1])==1):
#             nodes_list.pop()    
#         for nds in nodes_list:
#             final_list.append([1, isOneway,  category,nds])
junctions = set()       
for k in segMap.keys():
    if segMap[k] > 1:
        junctions.add(k)
    
print len(junctions)
df = pd.DataFrame(final_list,columns=['id','oneway','category','nodes'])
df

37589


,id,oneway,category,nodes
0,1,False,residential,"[115443169, 1930275790, 115443141, 115443115, ..."
1,1,False,residential,"[1930275776, 119891632, 115442946, 115443077, ..."
2,1,False,residential,"[614391691, 73569654, 88047791, 100115120, 100..."
3,1,False,residential,"[61104104, 101102967, 88082282, 66908824, 6690..."
4,1,True,primary,"[62542556, 1768006193, 558640325, 1768006012, ..."
5,1,True,primary_link,"[62550447, 62551805]"
6,1,True,trunk_link,"[62901951, 4375169071, 3559962135]"
7,1,True,trunk,"[71121979, 1721531798, 1721531811, 1721531799,..."
8,1,False,residential,"[824481103, 62924733, 3814803442, 94820531]"
9,1,False,residential,"[94820531, 3814803442, 62924733, 824481103]"


In [8]:
final_list=[]
for index, row in df.iterrows():
#     print index, row['nodes']
    st = 0;
    node_list = []
    for i in range(1, len(row['nodes']) - 1):
        if row['nodes'][i] in junctions:
            node_list.append(row['nodes'][st:i+1])
            st = i
    node_list.append(row['nodes'][st:len(row['nodes'])])
    for nst in node_list:
        final_list.append([1, row['oneway'], row['category'], nst ]) 
df = pd.DataFrame(final_list,columns=['id','oneway','category','nodes'])
df

,id,oneway,category,nodes
0,1,False,residential,"[115443169, 1930275790]"
1,1,False,residential,"[1930275790, 115443141, 115443115]"
2,1,False,residential,"[115443115, 115443077]"
3,1,False,residential,"[115443077, 115442946, 119891632]"
4,1,False,residential,"[119891632, 1930275776]"
5,1,False,residential,"[1930275776, 119891632]"
6,1,False,residential,"[119891632, 115442946, 115443077]"
7,1,False,residential,"[115443077, 115443115]"
8,1,False,residential,"[115443115, 115443141, 1930275790]"
9,1,False,residential,"[1930275790, 115443169]"


In [9]:
df['id']=df.index
df['nodes']=df['nodes'].apply(lambda x:list(x))
df

,id,oneway,category,nodes
0,0,False,residential,"[115443169, 1930275790]"
1,1,False,residential,"[1930275790, 115443141, 115443115]"
2,2,False,residential,"[115443115, 115443077]"
3,3,False,residential,"[115443077, 115442946, 119891632]"
4,4,False,residential,"[119891632, 1930275776]"
5,5,False,residential,"[1930275776, 119891632]"
6,6,False,residential,"[119891632, 115442946, 115443077]"
7,7,False,residential,"[115443077, 115443115]"
8,8,False,residential,"[115443115, 115443141, 1930275790]"
9,9,False,residential,"[1930275790, 115443169]"


In [10]:
df.to_csv('highway.csv',index=False)